# Crawl dữ liệu

In [2]:
import os
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup

# Khởi tạo cloudscraper để vượt qua Cloudflare
scraper = cloudscraper.create_scraper()

# Base URL của OKX (trang 1 là base URL, các trang sau theo dạng /page/{số trang})
base_url = "https://www.okx.com/vi/markets/prices"
all_data = []

# Lặp qua 12 trang
for page in range(1, 13):
    if page == 1:
        url = base_url
    else:
        url = base_url + "/page/" + str(page)
    print(f"🔄 Đang tải trang {page}: {url}")
    
    response = scraper.get(url, timeout=10)
    if response.status_code != 200:
        print(f"❌ Lỗi {response.status_code} khi tải trang {url}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    
    # Tìm table chứa dữ liệu (theo class của table trên OKX)
    table = soup.find("table", class_="table-box token-list-table")
    if not table:
        print(f"❌ Không tìm thấy table trên trang {page}")
        continue
    
    # Lấy header từ thead nếu có
    thead = table.find("thead")
    headers = []
    if thead:
        header_row = thead.find("tr")
        if header_row:
            headers = [th.get_text(strip=True) for th in header_row.find_all("th")]
    # Nếu không có header, định nghĩa header mặc định
    if not headers:
        headers = ["Tên", "Giá/Th.đổi (24 giờ)"]
    
    tbody = table.find("tbody")
    if not tbody:
        print(f"❌ Không tìm thấy tbody trên trang {page}")
        continue
    rows = tbody.find_all("tr")
    
    for row in rows:
        cols = row.find_all("td")
        row_data = [col.get_text(strip=True) for col in cols]
        all_data.append(row_data)

# Tạo DataFrame từ dữ liệu đã thu thập
df = pd.DataFrame(all_data, columns=headers)
# Nếu có cột trùng lặp (ví dụ, do trang có header lặp), loại bỏ chúng:
df = df.loc[:, ~df.columns.duplicated()]

# Lưu DataFrame ra file CSV (encoding để hỗ trợ tiếng Việt)
csv_path = os.path.join(os.getcwd(), "okx_prices_all_pages.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"✅ Dữ liệu đã được lưu vào {csv_path}")


🔄 Đang tải trang 1: https://www.okx.com/vi/markets/prices
🔄 Đang tải trang 2: https://www.okx.com/vi/markets/prices/page/2
🔄 Đang tải trang 3: https://www.okx.com/vi/markets/prices/page/3
🔄 Đang tải trang 4: https://www.okx.com/vi/markets/prices/page/4
🔄 Đang tải trang 5: https://www.okx.com/vi/markets/prices/page/5
🔄 Đang tải trang 6: https://www.okx.com/vi/markets/prices/page/6
🔄 Đang tải trang 7: https://www.okx.com/vi/markets/prices/page/7
🔄 Đang tải trang 8: https://www.okx.com/vi/markets/prices/page/8
🔄 Đang tải trang 9: https://www.okx.com/vi/markets/prices/page/9
🔄 Đang tải trang 10: https://www.okx.com/vi/markets/prices/page/10
🔄 Đang tải trang 11: https://www.okx.com/vi/markets/prices/page/11
🔄 Đang tải trang 12: https://www.okx.com/vi/markets/prices/page/12
✅ Dữ liệu đã được lưu vào d:\E\2024.2\khdl (DS)\weekly\DataScience\weekly_exercise\ex1_crawlData\beautifulSoup\OKX\okx_prices_all_pages.csv


# Thống kê về dữ liệu đã thu thập

In [3]:
# Đọc dữ liệu từ file CSV (đảm bảo encoding phù hợp)
df = pd.read_csv("okx_prices_all_pages.csv", encoding="utf-8-sig")

Hiển thị 5 dòng đầu tiên

In [4]:
print("5 dòng đầu tiên:")
print(df.head())

5 dòng đầu tiên:
           Tên Giá/Th.đổi (24 giờ)
0   BTCBitcoin    $91.904,10+1,43%
1  ETHEthereum     $2.297,38+2,49%
2    XRPRipple       $2,5104+0,42%
3   USDTTether      $0,99970+0,01%
4       BNBBNB       $605,50+1,19%


Hiển thị thông tin tổng quát của DataFrame

In [5]:
print("\nThông tin DataFrame:")
df.info()


Thông tin DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tên                  346 non-null    object
 1   Giá/Th.đổi (24 giờ)  346 non-null    object
dtypes: object(2)
memory usage: 5.5+ KB


Thống kê mô tả cơ bản (cho các cột số)

In [6]:
print("\nThống kê mô tả:")
print(df.describe())


Thống kê mô tả:
                   Tên Giá/Th.đổi (24 giờ)
count              346                 346
unique             346                 312
top     BETHBeacon ETH                $0--
freq                 1                  35
